学习率调整  
学习率控制更新的步伐

class_LRScheduler  
主要属性：  
* optimizer:关联的优化器  
* last_epoch:记录epoch数  
* base_lrs:记录初始学习率  

主要方法：  
* step():更新下一个epoch的学习率  
* get_lr():虚函数，计算下一个epoch的学习率  

scheduler.step要放到epoch的循环中去，不要放到iteration中去


In [16]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
torch.manual_seed(1)
LR = 0.1 
iteration = 10
max_epoch = 200

In [17]:
weights = torch.randn((1), requires_grad=True)
target = torch.zeros((1))
optimizer = optim.SGD([weights], lr=LR, momentum=0.9)

学习率调整策略  
1.StepLR  
功能：等间隔调整学习率  
主要参数：  
* step_size:调整间隔数  
* gamma:调整系数  

调整方式:lr = lr * gamma

In [19]:
scheduler_lr = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
lr_list, epoch_list = list(), list()
for epoch in range(max_epoch):
    lr_list.append(scheduler_lr.get_lr())
    epoch_list.append(epoch)
    for i in range(iteration):
        loss = torch.pow((weights - target), 2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler_lr.step()
plt.plot(epoch_list, lr_list, label='Step LR Scheduler')
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.legend()
plt.show()

2.MultiStepLR  
功能：按给定间隔调整学习率  
主要参数：  
* milestones:设定调整时刻数  
* gamma:调整系数  

调整方式:lr = lr * gamma

In [20]:
milestones = [50, 125, 160]
scheduler_lr = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
lr_list, epoch_list = list(), list()
for epoch in range(max_epoch):
    lr_list.append(scheduler_lr.get_lr())
    epoch_list.append(epoch)
    for i in range(iteration):
        loss = torch.pow((weights - target), 2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler_lr.step()
plt.plot(epoch_list, lr_list, label='Multi Step LR Scheduler\nmilestones:{}'.format(milestones))
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.legend()
plt.show()

3.ExponentialLR  
功能：按指数衰减调整学习率  
主要参数：  
* gamma:指数的底  

调整方式:lr = lr * gamma \** epoch

In [22]:
scheduler_lr = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
lr_list, epoch_list = list(), list()
for epoch in range(max_epoch):
    lr_list.append(scheduler_lr.get_lr())
    epoch_list.append(epoch)
    for i in range(iteration):
        loss = torch.pow((weights - target), 2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler_lr.step()
plt.plot(epoch_list, lr_list, label='Exponentila LR Scheduler\ngamma:{}'.format(0.95))
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.legend()
plt.show()

4.CosineAnnealingLR  
功能：余弦周期调整学习率  
主要参数：  
* T_max:下降周期  
* eta_min:学习率下限

In [23]:
t_max = 50
scheduler_lr = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max, eta_min=0.)
lr_list, epoch_list = list(), list()
for epoch in range(max_epoch):
    lr_list.append(scheduler_lr.get_lr())
    epoch_list.append(epoch)
    for i in range(iteration):
        loss = torch.pow((weights - target), 2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler_lr.step()
plt.plot(epoch_list, lr_list, label='CosineAnnealingLR Scheduler\nT_max:{}'.format(t_max))
plt.xlabel('Epoch')
plt.ylabel('Learning rate')
plt.legend()
plt.show()

5.ReduceLRonPlateau  
功能：监控指标，当指标不再变化则调整  
主要参数：  
* mode:min/max两种模式  
* factor:调整系数  
* patience:"耐心"，接受几次不变化  
* cooldown:"冷却时间"，停止监控一段时间  
* verbose:是否打印日志  
* min_lr:学习率下限  
* eps:学习率衰减最小值

In [24]:
loss_value = 0.5
accuracy = 0.9
factor = 0.1
mode='min'
patience = 10
cooldown=10
min_lr = 1e-4
verbose = True
scheduler_lr = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, mode=mode, patience=patience, cooldown=cooldown, min_lr=min_lr, verbose=True)
for epoch in range(max_epoch):
    for i in range(iteration):
        optimizer.step()
        optimizer.zero_grad()
    if epoch == 5:
        loss_value = 0.4
    scheduler_lr.step(loss_value)

Epoch    17: reducing learning rate of group 0 to 1.0000e-02.
Epoch    38: reducing learning rate of group 0 to 1.0000e-03.
Epoch    59: reducing learning rate of group 0 to 1.0000e-04.
